<a href="https://colab.research.google.com/github/Tomawock/MLDM_COVID-19/blob/master/testing/covid_random_forest_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing per le Random Forest

In [36]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import joblib
from datetime import datetime

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold
from sklearn.metrics import r2_score, mean_squared_error, max_error
from sklearn.ensemble import RandomForestRegressor 

from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

**Nomenclatura per i test**

In [37]:
path='/content/'
datasets_names= ['dataset_finale_medie_rnd1.csv','dataset_finale_medie_2_week_rnd1.csv','dataset_finale_medie_ponderate_rnd1.csv',
               'dataset_finale_medie_rnd220.csv','dataset_finale_medie_2_week_rnd220.csv','dataset_finale_medie_ponderate_rnd220.csv']

rdn_states= [1, 22, 777, 6654, 432145]
features_prediction= ['fut1_nuovi_positivi','fut2_nuovi_positivi',
                     'fut1_ricoverati_con_sintomi','fut2_ricoverati_con_sintomi',
                     'fut1_terapia_intensiva','fut2_terapia_intensiva',
                     'fut1_deceduti','fut2_deceduti',
                     'fut1_totale_ospedalizzati','fut2_totale_ospedalizzati',
                     'fut1_dimessi_guariti','fut2_dimessi_guariti']

coefficents=['R2','RMSE','MAXERR']

**Creazione indici per il dataset comune nel quale verranno messi i dati dei test**


In [38]:
col=[]
inde=[]
for feature in features_prediction:
  for coeff in coefficents:
    col.append(coeff+'_'+feature)
    #print(f"Columns_names\t{coeff+'_'+feature}")

for rnd in rdn_states:
  for path in datasets_names:
    inde.append(path+':'+str(rnd))
    #print(f"indexes_\t{path+':'+str(rnd)}")

df_results=pd.DataFrame(columns=col,index=inde)
print(df_results.columns)
print(df_results.index)

Index(['R2_fut1_nuovi_positivi', 'RMSE_fut1_nuovi_positivi',
       'MAXERR_fut1_nuovi_positivi', 'R2_fut2_nuovi_positivi',
       'RMSE_fut2_nuovi_positivi', 'MAXERR_fut2_nuovi_positivi',
       'R2_fut1_ricoverati_con_sintomi', 'RMSE_fut1_ricoverati_con_sintomi',
       'MAXERR_fut1_ricoverati_con_sintomi', 'R2_fut2_ricoverati_con_sintomi',
       'RMSE_fut2_ricoverati_con_sintomi',
       'MAXERR_fut2_ricoverati_con_sintomi', 'R2_fut1_terapia_intensiva',
       'RMSE_fut1_terapia_intensiva', 'MAXERR_fut1_terapia_intensiva',
       'R2_fut2_terapia_intensiva', 'RMSE_fut2_terapia_intensiva',
       'MAXERR_fut2_terapia_intensiva', 'R2_fut1_deceduti',
       'RMSE_fut1_deceduti', 'MAXERR_fut1_deceduti', 'R2_fut2_deceduti',
       'RMSE_fut2_deceduti', 'MAXERR_fut2_deceduti',
       'R2_fut1_totale_ospedalizzati', 'RMSE_fut1_totale_ospedalizzati',
       'MAXERR_fut1_totale_ospedalizzati', 'R2_fut2_totale_ospedalizzati',
       'RMSE_fut2_totale_ospedalizzati', 'MAXERR_fut2_totale_osped

In [ ]:
df_2 = pd.read_csv(datasets_path[0], parse_dates=["Data"], infer_datetime_format=True)
rnd_state = rdn_states[1]
predictor_columns = [
       'pass_Ammoniaca', 
       'pass_Benzene',
       'pass_Biossido di Azoto', 
       'pass_Biossido di Zolfo',
       'pass_Monossido di Azoto', 
       'pass_Monossido di Carbonio',
       'pass_Ossidi di Azoto', 
       'pass_Ozono', 
       'pass_PM10 (SM2005)',
       'pass_Particelle sospese PM2.5',
       'pass_Radiazione Globale', 
       'pass_Temperatura',
       'pass_deceduti',
       'pass_nuovi_positivi',
       'pass_ricoverati_con_sintomi', 
       'pass_tamponi']
################################################################################
train_2 = df_2.sample(frac=0.70, random_state=rnd_state)
test_2 = df_2.drop(train_2.index)

train_X_2 = train_2[predictor_columns]
train_y_2 = train_2[features_prediction[0]]

test_X_2 = test_2[predictor_columns]
test_y_2 = test_2[features_prediction[0]]
################################################################################
forest_regressor = RandomForestRegressor(bootstrap=False,random_state=rnd_state) 
################################################################################
imp=SimpleImputer(missing_values=np.nan, strategy="mean")
imp=imp.fit(train_X_2)
################################################################################
n_estimators_list = [100]
criterion_list = ["mse", "mae"]
max_depth_list = [None, 5, 6]
min_samples_split_list = [2, 3]
min_samples_split_leaf_list = [3,5]
min_weight_fraction_leaf_list = [0.0, 0.1]
max_features_list = ["auto", "sqrt", "log2"]
max_leaf_nodes_list = [None, 10, 25]
min_impurity_decrease_list = [0.0, 0.1]

param_grd = { "n_estimators":n_estimators_list,
              "criterion": criterion_list,
              "max_depth": max_depth_list,
              "min_samples_split": min_samples_split_list,
              "min_samples_leaf": min_samples_split_leaf_list,
              "min_weight_fraction_leaf": min_weight_fraction_leaf_list,
              "max_features": max_features_list,
              "max_leaf_nodes": max_leaf_nodes_list,
              "min_impurity_decrease": min_impurity_decrease_list}
################################################################################
grid_regressor_2 = GridSearchCV(forest_regressor, param_grd, 
                              n_jobs=-1, 
                              verbose=0)
################################################################################
grid_regressor_2.fit(imp.transform(train_X_2), train_y_2)
################################################################################
best_regressor_2 = grid_regressor_2.best_estimator_

imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
imputer = imputer.fit(test_X_2)
test_X_imp = imputer.transform(test_X_2)

predicted_2 = best_regressor_2.predict(test_X_imp)
################################################################################
r2=float("{:.3f}".format(r2_score(test_y_2, predicted_2)))
rmse=float("{:.3f}".format(mean_squared_error(test_y_2, predicted_2, squared=False)))
merr=float("{:.3f}".format(max_error(test_y_2, predicted_2)))
print(f"R2: \t\t{r2}")
print(f"RMSE: \t\t{rmse}")
print(f"MAX ERR: \t{merr}")
################################################################################

In [ ]:
# train_2 = df_2.sample(frac=0.70, random_state=rnd_state)
# test_2 = df_2.drop(train_2.index)

# train_X_2 = train_2[predictor_columns]
# train_y_2 = train_2[features_prediction[0]]

# test_X_2 = test_2[predictor_columns]
# test_y_2 = test_2[features_prediction[0]]

In [ ]:
# # create regressor object 
# forest_regressor = RandomForestRegressor(bootstrap=False,random_state=rnd_state) 

In [ ]:
# imp=SimpleImputer(missing_values=np.nan, strategy="mean")
# imp=imp.fit(train_X_2)

In [ ]:
# n_estimators_list = [100]
# criterion_list = ["mse", "mae"]
# max_depth_list = [None, 5, 6]
# min_samples_split_list = [2, 3]
# min_samples_split_leaf_list = [3,5]
# min_weight_fraction_leaf_list = [0.0, 0.1]
# max_features_list = ["auto", "sqrt", "log2"]
# max_leaf_nodes_list = [None, 10, 25]
# min_impurity_decrease_list = [0.0, 0.1]

# param_grd = { "n_estimators":n_estimators_list,
#               "criterion": criterion_list,
#               "max_depth": max_depth_list,
#               "min_samples_split": min_samples_split_list,
#               "min_samples_leaf": min_samples_split_leaf_list,
#               "min_weight_fraction_leaf": min_weight_fraction_leaf_list,
#               "max_features": max_features_list,
#               "max_leaf_nodes": max_leaf_nodes_list,
#               "min_impurity_decrease": min_impurity_decrease_list}

In [ ]:
# grid_regressor_2 = GridSearchCV(forest_regressor, param_grd, 
#                               n_jobs=-1, 
#                               verbose=1)

In [ ]:
# grid_regressor_2.fit(imp.transform(train_X_2), train_y_2)

In [ ]:
# best_regressor_2 = grid_regressor_2.best_estimator_

# imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
# imputer = imputer.fit(test_X_2)
# test_X_imp = imputer.transform(test_X_2)

# predicted_2 = best_regressor_2.predict(test_X_imp)

In [ ]:
# best_regressor_2

In [ ]:
# r2=float("{:.3f}".format(r2_score(test_y_2, predicted_2)))
# rmse=float("{:.3f}".format(mean_squared_error(test_y_2, predicted_2, squared=False)))
# merr=float("{:.3f}".format(max_error(test_y_2, predicted_2)))
# # print(f"R2: \t\t{r2}")
# # print(f"RMSE: \t\t{rmse}")
# # print(f"MAX ERR: \t{merr}")